# Coursework 2: Data Processing

## Task 1
This coursework will assess your understanding of using NoSQL to store and retrieve data.  You will perform operations on data from the Enron email dataset in a MongoDB database, and write a report detailing the suitability of different types of databases for data science applications.  You will be required to run code to answer the given questions in the Jupyter notebook provided, and write a report describing alternative approaches to using MongoDB.

Download the JSON version of the Enron data (using the “Download as zip” to download the data file from http://edshare.soton.ac.uk/19548/, the file is about 380MB) and import into a collection called messages in a database called enron.  You do not need to set up any authentication.  In the Jupyter notebook provided, perform the following tasks, using the Python PyMongo library.

Answers should be efficient in terms of speed.  Answers which are less efficient will not get full marks.

In [15]:
import pymongo
from pymongo import MongoClient
from datetime import datetime
from pprint import pprint

In [14]:
%%bash
head -n 100000 messages.json > messages_short.json
mongoimport --db enron --collection messages --drop --file messages_short.json

2019-12-12T16:36:09.716+0000	connected to: localhost
2019-12-12T16:36:09.716+0000	dropping: enron.messages
2019-12-12T16:36:12.715+0000	[##......................] enron.messages	43.2MB/354MB (12.2%)
2019-12-12T16:36:15.720+0000	[#####...................] enron.messages	87.2MB/354MB (24.6%)
2019-12-12T16:36:18.715+0000	[#########...............] enron.messages	136MB/354MB (38.5%)
2019-12-12T16:36:21.715+0000	[############............] enron.messages	187MB/354MB (52.8%)
2019-12-12T16:36:24.716+0000	[################........] enron.messages	238MB/354MB (67.4%)
2019-12-12T16:36:27.723+0000	[###################.....] enron.messages	284MB/354MB (80.2%)
2019-12-12T16:36:30.719+0000	[######################..] enron.messages	329MB/354MB (92.9%)
2019-12-12T16:36:32.687+0000	[########################] enron.messages	354MB/354MB (100.0%)
2019-12-12T16:36:32.687+0000	imported 100000 documents


### 1)
Write a function which returns a MongoDB connection object to the "messages" collection. [4 points] 

In [16]:
def get_collection():
    """
    Connects to the server, and returns a collection object
    of the `messages` collection in the `enron` database
    """
    # YOUR CODE HERE    
    #client = MongoClient('mongodb://localhost:127.0.1')
    client = MongoClient('mongodb://localhost:27017')
    #print client.list_database_names()
    db = client.enron
    collection = db.messages
    return collection

In [17]:
msg = get_collection()
d = msg.find().limit(10)
for a in d:
    pprint(a)


{u'_id': ObjectId('4f16fc97d1e2d32371003e27'),
 u'body': u"the scrimmage is still up in the air...\n\n\nwebb said that they didnt want to scrimmage...\n\nthe aggies  are scrimmaging each other... (the aggie teams practiced on \nSunday)\n\nwhen I called the aggie captains to see if we could use their field.... they \nsaid that it was tooo smalll for us to use...\n\n\nsounds like bullshit to me... but what can we do....\n\n\nanyway... we will have to do another practice Wed. night....    and I dont' \nknow where we can practice.... any suggestions...\n\n\nalso,  we still need one  more person...",
 u'filename': u'450.',
 u'headers': {u'Content-Transfer-Encoding': u'7bit',
              u'Content-Type': u'text/plain; charset=us-ascii',
              u'Date': u'Tue, 14 Nov 2000 08:22:00 -0800 (PST)',
              u'From': u'michael.simmons@enron.com',
              u'Message-ID': u'<6884142.1075854677416.JavaMail.evans@thyme>',
              u'Mime-Version': u'1.0',
              u'Subjec

### 2)

Write a function which returns the amount of emails in the messages collection in total. [4 points] 

In [18]:
def get_amount_of_messages(collection1):
    """
    :param collection A PyMongo collection object
    :return the amount of documents in the collection
    """    
    # YOUR CODE HERE
    num = collection1.count_documents({})
    
    return num
    
get_amount_of_messages(msg)

100000

### 3) 

Write a function which returns each person who was BCCed on an email.  Include each person only once, and display only their name according to the X-To header. [4 points] 



In [19]:
import re
def get_bcced_people(collection):
    """
    :param collection A PyMongo collection object
    :return the names of the people who have received an email by BCC
    """    
    # YOUR CODE HERE
    Name= []
    #filter = {"headers.X-bcc":{'$exists':True}};
    #filter = {"headers.X-bcc":{'$ne':''}};
    filter = {"$and": [{"headers.X-bcc":{'$exists':True}}, {"headers.X-bcc":{'$ne':''}}]}
    
    fields = {'_id': False, 'headers.X-bcc': True}
    name = collection.find(filter, fields)
    #for n in name:
    #    names = name.split(",")
    #    Name.appemd(names)
    #for i in Name:
    #    print(i)
    
    
#     for n in name:
#         pprint(n) 
    Name_list = []        
    for str in name:
        #pprint(str)
        string = str['headers']['X-bcc']
        Name = re.split(r'<.+?>', string)
        for Name2 in Name:
            #print Name2
            Name3 = Name2.strip(', ')        
            #print(Name3)
            Name_list.append(Name3)
        
    result = list(set(Name_list))
    pprint(result)
    
    pass

get_bcced_people(msg)

[u'',
 u'Nettelton, Marcus',
 u'Apollo, Beth',
 u"'Martin.W.Penkwitz@enron.com', Berg, Mary",
 u'Guinn, Linda',
 u'Wood, Kim',
 u'Evans, Mark',
 u'Susan J Mara',
 u'Delahay, Julie',
 u'Cole, Kate',
 u"'lpinder@enron.com', 'lstoler@enron.com', Watanabe, Luiz",
 u"'jrigby@enron.com', 'jtalcott@enron.com', Chin, Julia H.",
 u'Guerrero, Janel',
 u"'kathleen_magruder@enron.com', Mann, Kay",
 u'Barry, Patrick',
 u'Boyd, Justin',
 u'Choate, Heather',
 u'Robertson, Audrey',
 u'Mellencamp, Lisa',
 u"'mkeyser@enron.com', 'mmalmeida@elektro.com.br', 'mmanly@newpower.com', 'mmaxwel@enron.com', Maynard, Marc",
 u"'hfromer@enron.com', Magual, Ibrahim",
 u'Beth Apollo',
 u"'jhartso@enron.com', 'jhillego@enron.com', 'jhmoore@enron.com', Armogida, Jim",
 u"'cameron@perfect.com'",
 u'Perry, Renee',
 u'Ogenyi, Gloria',
 u'Schwartzenburg, John',
 u'Azevedo, Karla',
 u'Wehring, Linda',
 u'Young, Kay',
 u'Pardue, Larry',
 u"'michael.moran@enron.com', Schuh, Michael",
 u'Robichaux, Lisa',
 u"'jale@azurix.com

### 4)

Write a function with parameter subject, which gets all emails in a thread with that parameter, and orders them by date (ascending). “An email thread is an email message that includes a running list of all the succeeding replies starting with the original email.”, check for detail descriptions at https://www.techopedia.com/definition/1503/email-thread [4 points]

In [20]:
sub = 'Plays and '
def get_emails_in_thread(collection, subject):
    """
    :param collection A PyMongo collection object
    :return All emails in the thread with that subject
    """    
    # YOUR CODE HERE   
    match = {
        "$match": {"headers.Subject": {"$regex": subject}}
    }
    cursor = collection.aggregate([match])
#     pprint (list(cursor))
    date1 = {}
    for date in cursor:
        Datetime = datetime.strptime(date['headers']['Date'][5:-12], "%d %b %Y %H:%M:%S")
        date1.update({Datetime: date})
        
#     pprint(date1)
#     pprint (sorted(date1, reverse=True))
    return sorted(date1.items(), key=lambda k: k[0])
    
get_emails_in_thread(msg, sub)

[(datetime.datetime(2000, 11, 14, 4, 17),
  {u'_id': ObjectId('4f16fc98d1e2d32371004365'),
   u'body': u"I know I might seem a little obsessed with this, but here are some of our \nplays from our flag football league.  There are plays on both tabs.\nI was thinking that we would start with me at left flanker, Paul Broderick at \ncenter, Paul Burkhart at right slot, Steve at right flanker, Sarah at \nblocker, and either Mike or Chris at blocker.  Let me know if you have \nobjections.  Also, I am sending this to Paul Burkhart so I am assuming that \nhe will play (P. Broderick can you confirm this?).\n\nSteve - if you have some plays from your team that you think might work let \nme know.\n\nLet me know what you think.\n\nHook 'Em!\n-Eric\n\n\n",
   u'filename': u'77.',
   u'headers': {u'Content-Transfer-Encoding': u'7bit',
    u'Content-Type': u'text/plain; charset=us-ascii',
    u'Date': u'Tue, 14 Nov 2000 04:17:00 -0800 (PST)',
    u'From': u'eric.bass@enron.com',
    u'Message-ID': u'<

### 5)

Write a function which returns the percentage of emails sent on a weekend (i.e., Saturday and Sunday) as a `float` between 0 and 1. [6 points]

In [21]:
def get_percentage_sent_on_weekend(collection):
    """
    :param collection A PyMongo collection object
    :return A float between 0 and 1
    """    
    # YOUR CODE HERE
    filter = {"$or":[{"headers.Date": {"$regex": '^Sat'}},{"headers.Date": {"$regex": '^Sun'}}]}    
    filter2 = collection.find(filter)
    #for weekend in msg.find(filter):
    #    pprint(weekend)
    #num = filter2.count_documents()
    num = filter2.count()
    per = float(num)/get_amount_of_messages(collection)
    return per
get_percentage_sent_on_weekend(msg)

/home/comp6235/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if sys.path[0] == '':


0.0393

### 6)

Write a function with parameter limit. The function should return for each email account: the number of emails sent, the number of emails received, and the total number of emails (sent and received). Use the following format: [{"contact": "michael.simmons@enron.com", "from": 42, "to": 92, "total": 134}] and the information contained in the To, From, and Cc headers. Sort the output in descending order by the total number of emails. Use the parameter limit to specify the number of results to be returned. If limit is null, the function should return all results. If limit is higher than null, the function should return the number of results specified as limit. limit cannot take negative values. [10 points]

In [22]:
from collections import defaultdict
from collections import OrderedDict
def get_emails_between_contacts(collection, limit):
    """
    Shows the communications between contacts
    Sort by the descending order of total emails using the To, From, and Cc headers.
    :param `collection` A PyMongo collection object    
    :param `limit` An integer specifying the amount to display, or
    if null will display all outputs
    :return A list of objects of the form:
    [{
        'contact': <<Another email address>>
        'from': 
        'to': 
        'total': 
    },{.....}]
    """    
    # YOUR CODE HERE

    filter_From = {
    '$match': {"$and": [{"headers.From":{'$exists':True}}, {"headers.From":{'$ne':''}}]}
    }
    filter_Cc = {
    '$match': {"$and": [{"headers.Cc":{'$exists':True}}, {"headers.Cc":{'$ne':''}}]}
    }
    filter_To = {
    '$match': {"$and": [{"headers.To":{'$exists':True}}, {"headers.To":{'$ne':''}}]}
    }
    
    Total = defaultdict(lambda: 0)
    From_cal = defaultdict(lambda: 0)
    To_cal = defaultdict(lambda: 0)
    Cc_cal = defaultdict(lambda: 0)
    
    group1 = {
    '$group': {
        '_id': '$headers.From', 
        'size': {'$sum': 1},
        }
    }
    limits = {
        '$limit': 10
    }
    FromEmails = list(collection.aggregate([filter_From, group1]))
#     pprint(FromEmails)
    
    
    for Spl_FromEmails in FromEmails:
        Spl_FromEmails2 = Spl_FromEmails['_id'].split(",")
        for Spl_FromEmails3 in Spl_FromEmails2:
            Spl_FromEmails4 = Spl_FromEmails3.strip()
            From_cal[Spl_FromEmails4] = From_cal[Spl_FromEmails4] + Spl_FromEmails['size']
            Total[Spl_FromEmails4] = Total[Spl_FromEmails4] + Spl_FromEmails['size']
    
    
    group2 = {
    '$group': {
        '_id': '$headers.To', 
        'size': {'$sum': 1},
        }
    }
   
    ToEmails = list(collection.aggregate([filter_To, group2]))
#     pprint(ToEmails)
   
    
    for Spl_ToEmails in ToEmails:
        Spl_ToEmails2 = Spl_ToEmails['_id'].split(",")
        for Spl_ToEmails3 in Spl_ToEmails2:
            Spl_ToEmails4 = Spl_ToEmails3.strip()
            To_cal[Spl_ToEmails4] = To_cal[Spl_ToEmails4] + Spl_ToEmails['size']
            Total[Spl_ToEmails4] = Total[Spl_ToEmails4] + Spl_ToEmails['size']
#     To_cal = sorted(To_cal.items(), key=lambda k: k[1], reverse=True)    
    
    group3 = {
    '$group': {
        '_id': '$headers.Cc', 
        'size': {'$sum': 1},
        }
    }
    
    CcEmails = list(collection.aggregate([filter_Cc, group3]))    
    

#     pprint(CcEmails)
    
    for Spl_CcEmails in CcEmails:
        Spl_CcEmails2 = Spl_CcEmails['_id'].split(",")
        for Spl_CcEmails3 in Spl_CcEmails2:
            Spl_CcEmails4 = Spl_CcEmails3.strip()
            Cc_cal[Spl_CcEmails4] = Cc_cal[Spl_CcEmails4] + Spl_CcEmails['size']
            Total[Spl_CcEmails4] = Total[Spl_CcEmails4] + Spl_CcEmails['size']
#             print (Spl_CcEmails4, Spl_CcEmails['size'])

#     Cc_cal = sorted(Cc_cal.items(), key=lambda k: k[1], reverse=True)
    
    
        
    i = 1
#     pprint(sorted(Total.items(), key=lambda k: k[1], reverse=True))
    result = sorted(Total.items(), key=lambda k: k[1], reverse=True)
    for count in result:
        name = count[0]
        From_count = From_cal[name]
        To_count = To_cal[name]
        Cc_count = Cc_cal[name]
        if i > limit:
            break
#         print count
        i = i+1
        dict = {'contact':name, 'from':From_count, 'to':To_count,'total':count[1]}
#         print (dict)  #it can work but the sequence will be total, to contact from, so I change to OrderedDict
        od = OrderedDict()
        od['contact'] = name
        od['from'] = From_count
        od['to'] = To_count + Cc_count
        od['total'] = count[1]
        print od

#     pprint(sorted(Cc_cal.items(), key=lambda k: k[1], reverse=True))
    
    
        
#     for n in Total:
#         print (n)
        
        
    pass
get_emails_between_contacts(msg,10)

OrderedDict([('contact', u'jeff.dasovich@enron.com'), ('from', 9424), ('to', 12136), ('total', 21560)])
OrderedDict([('contact', u'sally.beck@enron.com'), ('from', 4244), ('to', 6905), ('total', 11149)])
OrderedDict([('contact', u'daren.farmer@enron.com'), ('from', 2812), ('to', 6275), ('total', 9087)])
OrderedDict([('contact', u'susan.mara@enron.com'), ('from', 819), ('to', 5127), ('total', 5946)])
OrderedDict([('contact', u'james.steffes@enron.com'), ('from', 473), ('to', 4802), ('total', 5275)])
OrderedDict([('contact', u'richard.shapiro@enron.com'), ('from', 202), ('to', 4672), ('total', 4874)])
OrderedDict([('contact', u'paul.kaufman@enron.com'), ('from', 197), ('to', 4501), ('total', 4698)])
OrderedDict([('contact', u'david.delainey@enron.com'), ('from', 2959), ('to', 758), ('total', 3717)])
OrderedDict([('contact', u'larry.campbell@enron.com'), ('from', 1380), ('to', 1894), ('total', 3274)])
OrderedDict([('contact', u'harry.kingerski@enron.com'), ('from', 106), ('to', 3074), ('t

### 7)
Write a function to find out the number of senders who were also direct receivers. Direct receiver means the email is sent to the person directly, not via cc or bcc. [4 points]

In [23]:
def get_from_to_people(collection):
    """
    :param collection A PyMongo collection object
    :return the NUMBER of the people who have sent emails and received emails as direct receivers.
    """    
    # YOUR CODE HERE
    send = collection.distinct("headers.From")
    receive = collection.distinct("headers.To")
    list_receive = []
    for sp in receive:
        sp2 = sp.split(",")
        for sp3 in sp2:
            sp4 = sp3.strip()
#             sp5 = sp4.strip(",\rn")
            
            #print sp5
            list_receive.append(sp4)
            
    result = len(list(set(send).intersection(list_receive)))
    return result

c = get_from_to_people(msg)
#print list(s)[:10]
print c

3411


### 8)
Write a function with parameters start_date and end_date, which returns the number of email messages that have been sent between those specified dates, including start_date and end_date [4 points] 

In [24]:
def get_emails_between_dates(collection, start_date, end_date):
    """
    :param collection A PyMongo collection object
    :return All emails between the specified start_date and end_date
    """    
    # YOUR CODE HERE
    
    group = {
    '$group': {
        '_id': "$headers.Date", 
        'size': {'$sum': 1},        
        }
    }
    limit = {
        '$limit': 50
    }
    Date = list(collection.aggregate([group]))
#     print (Date)
    num = 0
    for c in Date:
        if len(c['_id']) == 36 or len(c['_id']) == 37:
            d = datetime.strptime(c['_id'][5:-12], "%d %b %Y %H:%M:%S")
            if start < d and d < end:
#                 print d, c['size']   #This can show all of the results
                num = num + c['size']
    
    return num

start = datetime(2000,12,1,23,59)
end = datetime(2000,12,2,23,59)
get_emails_between_dates(msg, start, end)

16

## Task 2
This task will assess your ability to use the Hadoop Streaming API and MapReduce to process data. For each of the questions below, you are expected to write two python scripts, one for the Map phase and one for the Reduce phase. You are also expected to provide the correct parameters to the `hadoop` command to run the MapReduce process. Write down your answers in the specified cells below.

To get started, you need to download and unzip the YouTube dataset (available at http://edshare.soton.ac.uk/19547/) onto the machine where you have Hadoop installed (this should be the virtual machine provided).

To help you, `%%writefile` has been added to the top of the cells, automatically writing them to "mapper.py" and "reducer.py" respectively when the cells are run.

### 1) 
Using Youtube01-Psy.csv, find the hourly interval in which most spam was sent. The output should be in the form of a single key-value pair, where the value is a datetime at the start of the hour with the highest number of spam comments. [9 points]

In [25]:
%%bash

wget https://archive.ics.uci.edu/ml/machine-learning-databases/00380/YouTube-Spam-Collection-v1.zip \
-O YouTube-Spam-Collection-v1.zip

unzip -o YouTube-Spam-Collection-v1.zip

ls -lh *.csv

Archive:  YouTube-Spam-Collection-v1.zip
-rwxrwxrwx 1 comp6235 root 57K Mar 26  2017 Youtube01-Psy.csv
-rwxrwxrwx 1 comp6235 root 63K Mar 26  2017 Youtube02-KatyPerry.csv
-rwxrwxrwx 1 comp6235 root 63K Mar 26  2017 Youtube03-LMFAO.csv
-rwxrwxrwx 1 comp6235 root 81K Mar 26  2017 Youtube04-Eminem.csv
-rwxrwxrwx 1 comp6235 root 72K Mar 26  2017 Youtube05-Shakira.csv


--2019-12-12 16:38:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/00380/YouTube-Spam-Collection-v1.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘archive.ics.uci.edu’
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
note:  YouTube-Spam-Collection-v1.zip may be a plain executable, not an archive
unzip:  cannot find zipfile directory in one of YouTube-Spam-Collection-v1.zip or
        YouTube-Spam-Collection-v1.zip.zip, and cannot find YouTube-Spam-Collection-v1.zip.ZIP, period.


In [26]:
%%writefile mapper.py
#!/usr/bin/env python
#Answer for mapper.py

import csv
import sys
import re

lines = sys.stdin.readlines()

csvreader = csv.reader(lines)
# YOUR CODE GOES BELOW

# Create a list of ONLY the comments using a list comprehension
datetime = [row[2] for row in csvreader]

# Iterate over each of the comments
for datetimes in datetime:
    # Split the comment string into words, using every whitespace character as a divider. 
    tokens = datetimes.split(":", 1)[0]
    #tokens = re.split("T", datetimes)
    #tokens = re.sub(r"T", ' ', datetimes)
        #Print the key, value pair <token, 1>
    print(tokens + "\t1")




Overwriting mapper.py


In [27]:
%%writefile reducer.py
#!/usr/bin/env python
#Answer for reducer.py

import sys
from collections import defaultdict
# Keep simple example in for now, switch to stdin later

input_pairs = [
    '+447935454150	1',
    'lovely	1',
    'lovely	1',
    'girl	1',
    'talk	1',
    'to	1',
    'me	1'
    #'xxx	1',
     #Add an extra one to test that it works
    #'to\t1'
]
# Once we test this with streams, we can uncomment this next line
input_pairs = sys.stdin.readlines()

# YOUR CODE GOES BELOW

# Create a default dictionary. 
# This is a key-value store (dictionary) which returns a default value if the key hasn't been added.
# Here, we use it to store <word, count> pairs.
accumulator = defaultdict(lambda: 0)

for row in input_pairs:
    # Split the line into our key value pair.
    key_value_pair = row.split("\t", 1)
    
    # If we don't have a pair, ignore the line, as something has gone wrong.
    if len(key_value_pair) != 2:
        continue
        
    word = key_value_pair[0]
    # Strip removes whitespace at the start and end of a string. In this case, making sure we have just a number.
    # We also convert it to an integer here.
    count = int(key_value_pair[1].strip())
    
    # Retrieve the count of that word we've seen so far, add to it, then store the result.
    accumulator[word] = accumulator[word] + count
    
# for (key, value) in accumulator.items():
#     print(key + "\t" + str(value))

Most_Spam = max(accumulator.iteritems(), key = lambda k: k[1])
print("hour_with_most_spam\t\"%s\"" %(Most_Spam[0]+":00:00"))

Overwriting reducer.py


In [17]:
%%bash
chmod a+x mapper.py reducer.py
cat Youtube01-Psy.csv | ./mapper.py | ./reducer.py | sort

hour_with_most_spam	"2014-11-08T10:00:00"


In [28]:
%%bash

rm -rf output

hadoop-standalone-mode.sh

hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper.py,reducer.py \
-input Youtube01-Psy.csv \
-mapper ./mapper.py \
-reducer ./reducer.py \
-output output

Hadoop switched to standalone mode.


OpenJDK Server VM warning: You have loaded library /opt/hadoop-2.8.5/lib/native/libhadoop.so.1.0.0 which might have disabled stack guard. The VM will try to fix the stack guard now.
It's highly recommended that you fix the library with 'execstack -c <libfile>', or link it with '-z noexecstack'.
19/12/12 16:39:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
19/12/12 16:39:09 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
19/12/12 16:39:09 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
19/12/12 16:39:09 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
19/12/12 16:39:10 INFO mapred.FileInputFormat: Total input files to process : 1
19/12/12 16:39:10 INFO mapreduce.JobSubmitter: number of splits:1
19/12/12 16:39:10 INFO mapreduce.JobSubmitter: Submitting token

In [ ]:
#Expected key-value output format:
#hour_with_most_spam	"2013-11-10T10:00:00"

#Additional key-value pairs are acceptable, as long as the hour_with_most_spam pair is correct.

### 2) 
Find all comments associated with a username (the AUTHOR field). Return a JSON array of all comments associated with that username. (This should use the data from all 5 data files: Psy, KatyPerry, LMFAO, Eminem, Shakira) [11 points]

In [19]:
%%writefile mapper.py
#!/usr/bin/env python
#Answer for mapper.py
import csv
import sys
import re

lines = sys.stdin.readlines()

csvreader = csv.reader(lines)
# YOUR CODE GOES BELOW
username1 = ' '.join(sys.argv[1:])

# Create a list of ONLY the comments using a list comprehension

for row in csvreader:
    if row[1] == username1:
        print(row[1]+"\t"+row[3])



Overwriting mapper.py


In [20]:
%%writefile reducer.py
#!/usr/bin/env python
#Answer for reducer.py

import sys
from collections import defaultdict
# Keep simple example in for now, switch to stdin later

input_pairs = sys.stdin.readlines()
accumulator = defaultdict(list)

for row in input_pairs:
    # Split the line into our key value pair.
    key_value_pair = row.split("\t", 1)
    
    if len(key_value_pair) != 2:
        continue
        
    name = key_value_pair[0]
    # Strip removes whitespace at the start and end of a string. In this case, making sure we have just a number.
    # We also convert it to an integer here.
    comment = key_value_pair[1]
    
    # Retrieve the count of that word we've seen so far, add to it, then store the result.
    accumulator[name].append(comment)
    
for (key, value) in accumulator.items():
    print(key + "\t" + str(value))    

Overwriting reducer.py


In [15]:
%%bash
chmod a+x mapper.py reducer.py
cat Youtube01-Psy.csv | ./mapper.py | ./reducer.py | sort

hour_with_most_spam	"2014-11-08T10:00:00"


In [21]:
%%bash
username="Alex Martin"
User="./mapper.py $username"
#Hadoop command to run the map reduce.
rm -rf output
hadoop-standalone-mode.sh
hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper.py,reducer.py \
-input Youtube01-Psy.csv,Youtube02-KatyPerry.csv,Youtube03-LMFAO.csv,Youtube04-Eminem.csv,Youtube05-Shakira.csv \
-mapper "$User" \
-reducer ./reducer.py \
-output output

Hadoop switched to standalone mode.


OpenJDK Server VM warning: You have loaded library /opt/hadoop-2.8.5/lib/native/libhadoop.so.1.0.0 which might have disabled stack guard. The VM will try to fix the stack guard now.
It's highly recommended that you fix the library with 'execstack -c <libfile>', or link it with '-z noexecstack'.
19/12/12 15:25:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
19/12/12 15:25:59 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
19/12/12 15:25:59 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
19/12/12 15:25:59 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
19/12/12 15:25:59 INFO mapred.FileInputFormat: Total input files to process : 5
19/12/12 15:25:59 INFO mapreduce.JobSubmitter: number of splits:5
19/12/12 15:25:59 INFO mapreduce.JobSubmitter: Submitting token

In [ ]:
#Expected key-value output format:
#John Smith	["Comment 1", "Comment 2", "Comment 3", "etc."]
#Jane Doe	["Comment 1", "Comment 2", "Comment 3", "etc."]